<a href="https://colab.research.google.com/github/Muppasaisrija/Muppa_NFO5731_-SPRING2022/blob/main/In_class_exercise_04_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/29/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
!pip install gensim

In [ ]:
# Write your code here

#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import time
import logging
import argparse

#load csv file and create dataframe
df = pd.read_csv(r'C:\Users\admin\Downloads\archive (5)\tweet_data.csv')
df


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,���It felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20045,815757572,True,golden,259,NaN,female,1.0000,yes,1.0,8/5/15 21:16,...,https://pbs.twimg.com/profile_images/656793310...,0,C0DEED,"@lookupondeath ...Fine, and I'll drink tea too...",NaN,783,10/26/15 13:20,6.587400e+17,Verona ���,NaN
20046,815757681,True,golden,248,NaN,male,1.0000,yes,1.0,8/15/12 21:17,...,https://pbs.twimg.com/profile_images/639815429...,0,0,Greg Hardy you a good player and all but don't...,NaN,13523,10/26/15 12:40,6.587300e+17,"Kansas City, MO",NaN
20047,815757830,True,golden,264,NaN,male,1.0000,yes,1.0,9/3/12 1:17,...,https://pbs.twimg.com/profile_images/655473271...,0,C0DEED,You can miss people and still never want to se...,NaN,26419,10/26/15 13:20,6.587400e+17,Lagos Nigeria,NaN
20048,815757921,True,golden,250,NaN,female,0.8489,yes,1.0,11/6/12 23:46,...,https://pbs.twimg.com/profile_images/657716093...,0,0,@bitemyapp i had noticed your tendency to pee ...,NaN,56073,10/26/15 12:40,6.587300e+17,Texas Hill Country,NaN


In [ ]:
##Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

K = 10
def lda_clustering(df, K):
    #create a count matrix from the corpus
    count_vectorizer = CountVectorizer()
    count_matrix = count_vectorizer.fit_transform(df['text'])
    #use tfidf to normalize the word counts
    tfidf_transformer = TfidfTransformer()
    tfidf_matrix = tfidf_transformer.fit_transform(count_matrix)
    #compute the similarity between the documents
    #cosine_similarity(tfidf_matrix)
    #create a lda model with K topics
    lda = LatentDirichletAllocation(n_components=K, learning_method='online', learning_offset=50.,random_state=0).fit(tfidf_matrix)
    #get the topics for each document
    lda_output = lda.transform(tfidf_matrix)
    #get the top 10 words for each topic
    topic_word = lda.components_
    vocab = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(topic_word):
        print("Topic %d:" % (topic_idx))
        print(" ".join([vocab[i] for i in topic.argsort()[:-10 - 1:-1]]))
    #get the top 10 documents for each topic
    doc_topic = lda.transform(tfidf_matrix)
    for i, topic_dist in enumerate(doc_topic):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-10:-1]
        print('Document {}: {}'.format(i, ' '.join(topic_words)))
    #get the coherence score
    coherence_model_lda = CoherenceModel(model=lda, texts=df['text'], dictionary=count_vectorizer.vocabulary_, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)

    return lda_output, topic_word, doc_topic

lda_clustering(df, K)


In [ ]:

#Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics
#pip install gensim

from gensim import corpora, models
#the number of topics K should be decided by the coherence score of the topics
#the coherence score is the sum of the log of the probability of the words in the topic
K = 5
#Tokenize the sentence into words


#tokenazize the dataset
df['tokenized'] = df['text'].apply(lambda x: x.split())

#create a dictionary of the tokens
dictionary = df['tokenized'].apply(lambda x: list(set(x)))

#dictionary = corpora.Dictionary(df['Text'].split())
print(dictionary)
corpus = dictionary.apply(lambda x: list(dictionary.index)).apply(lambda x: list(zip(x, [1]*len(x))))


lda = models.LdaModel(corpus, num_topics=K, id2word=dictionary, passes=10)

def print_topics(model, top_n=10):
    for idx, topic in enumerate(model.top_topics(num_words=top_n)):
        print('Topic {}: {}'.format(idx, ' '.join([word for word, prop in topic[1]])))
print_topics(lda)
#then summarize what are the topics
lda.print_topics(num_topics=K, num_words=10)

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here

#Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics.
#import libraries
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
#import LSA
from sklearn.decomposition import TruncatedSVD
from gensim import corpora, models
#Generate K topics by using LSA, the number of topics K should be decided by the coherence score
K = 5
#the coherence score is the sum of the log of the probability of the words in the topic
lsa = TruncatedSVD(n_components=K, n_iter=10, random_state=42)
#create a pipeline to normalize the tf-idf vectors
pipeline = make_pipeline(TfidfVectorizer(), Normalizer(copy=False), lsa)
#fit the pipeline to the data
pipeline.fit(df['Text'])
#summarize what are the topics
print_topics(lsa)
#then summarize what are the topics
lsa.print_topics(num_topics=K, num_words=10)

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here
#Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics
# import ldavec
from gensim.models import LdaMulticore
from gensim.models.wrappers import LdaVec
#the number of topics K should be decided by the coherence score of the topics
#the coherence score is the sum of the log of the probability of the words in the topic
K = 5
#the number of iterations
iterations = 10
#the number of passes
passes = 10
lda2vec = LdaVec(corpus=corpus, num_topics=K, id2word=dictionary, passes=passes, iterations=iterations)
# summarize what are the topics
print_topics(lda2vec)
#then summarize what are the topics
lda2vec.print_topics(num_topics=K, num_words=10)

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here
#Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics
#import libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
# import BERTopic
from gensim.models import LdaMulticore
from gensim.models.wrappers import LdaVec
#the number of topics K should be decided by the coherence score of the topics

BERTopic = LdaVec(corpus=corpus, num_topics=K, id2word=dictionary, passes=10, iterations=10)
# summarize what are the topics
print_topics(BERTopic)
#then summarize what are the topics
BERTopic.print_topics(num_topics=K, num_words=10)


## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)
# Compare the results generated by the four topic modeling algorithms
"""
The results are similar.
"""
#which is better between lda,lsa?
"""
LSA.

in Natural Language Processing (NLP): Latent Semantic Analysis (LSA), 
Latent Dirichlet Allocation (LDA) and lexical chains. These techniques were evaluated 
and compared on two different corpora in order to highlight the similarities and differences 
between them from a semantic analysis viewpoint. The first corpus consisted of four Wikipedia 
articles on different topics, while the second one consisted of 35 online chat conversations 
between 4-12 participants debating four imposed topics (forum, chat, blog and wikis). The study 
focuses on finding similarities and differences between the outcomes of the three methods from a 
semantic analysis point of view, by computing quantitative factors such as correlations, degree of 
coverage of the resulting topics, etc. Using corpora from different types of discourse and quantitative 
factors that are task-independent allows us to prove that although LSA and LDA provide similar results,
the results of lexical chaining are not very correlated with neither the ones of LSA or LDA, therefore 
lexical chains might be used complementary to LSA or LDA when performing semantic analysis for various NLP applications.
These factors out to be the reason why the LSA are the better.
"""



